# 🤖 GPT-5 Multimodal - Analyse et Génération d'Images

**Module :** 01-Images-Foundation  
**Niveau :** 🟢 Débutant  
**Technologies :** GPT-5, OpenRouter API, Vision AI  
**Durée estimée :** 30 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Configurer GPT-5 via OpenRouter pour analyse d'images
- [ ] Maîtriser les conversations multimodales texte + image
- [ ] Analyser et décrire des images avec précision
- [ ] Créer des prompts optimisés pour l'analyse visuelle
- [ ] Intégrer GPT-5 dans des cas d'usage pédagogiques

## 📚 Prérequis

- Environment Setup (module 00) complété
- Clé API OpenRouter configurée
- Connaissances de base en IA multimodale

## ⚡ Capacités GPT-5

- **Vision avancée** : Analyse détaillée d'images
- **Multimodal** : Conversation texte + image simultanée
- **Contexte étendu** : Jusqu'à 200K tokens
- **Raisonnement** : Analyse complexe et déductive
- **Éducatif** : Parfait pour l'enseignement

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration conversation
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"               

# Paramètres GPT-5
model_name = "openai/gpt-5"        # Modèle via OpenRouter
max_tokens = 4000                  # Tokens de réponse max
temperature = 0.7                  # Créativité (0.0-1.0)
top_p = 0.9                        # Diversité sampling

# Configuration analyse
analysis_mode = "detailed"         # "quick", "detailed", "educational"
include_technical_details = True   # Détails techniques images
export_analysis = True             # Sauvegarder analyses
generate_alt_text = True           # Générer descriptions accessibilité

# Paramètres pédagogiques
educational_level = "university"   # "elementary", "secondary", "university"
language = "français"              # Langue des explications
include_examples = True            # Inclure exemples pratiques

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Union
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
import logging
from urllib.parse import urlparse

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging, load_genai_config
        print("✅ Helpers GenAI importés")
    except ImportError:
        print("⚠️  Helpers GenAI non disponibles - mode autonome")

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('gpt5_multimodal')

print(f"🤖 GPT-5 Multimodal - Analyse et Génération d'Images")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Mode: {notebook_mode}, Analyse: {analysis_mode}, Niveau: {educational_level}")
print(f"🌍 Langue: {language}, Max tokens: {max_tokens}")

In [ ]:
# Configuration API OpenRouter pour GPT-5
print("\n🔑 CONFIGURATION GPT-5 MULTIMODAL")
print("=" * 42)

# Vérification clé API
openrouter_key = os.getenv('OPENROUTER_API_KEY')
if not openrouter_key:
    raise ValueError("❌ OPENROUTER_API_KEY manquante dans .env")

print(f"✅ Clé API OpenRouter configurée")

# Configuration headers et endpoint
api_base_url = "https://openrouter.ai/api/v1"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "HTTP-Referer": "https://coursia.myia.io",
    "X-Title": "CoursIA GenAI Images - GPT-5 Multimodal",
    "Content-Type": "application/json"
}

# Test connexion et vérification modèle GPT-5
try:
    response = requests.get(f"{api_base_url}/models", headers=headers, timeout=10)
    if response.status_code == 200:
        models_data = response.json()
        gpt5_models = [m for m in models_data.get('data', []) if 'gpt-5' in m.get('id', '').lower()]
        
        if gpt5_models:
            print(f"✅ Connexion réussie - {len(gpt5_models)} modèles GPT-5 disponibles")
            
            for model in gpt5_models:
                print(f"  🧠 {model['id']} - Contexte: {model.get('context_length', 'N/A')} tokens")
                if 'vision' in model.get('capabilities', []):
                    print(f"     👁️  Capacités vision activées")
        else:
            print(f"⚠️  Aucun modèle GPT-5 détecté - vérifiez votre accès")
            print(f"🔍 Modèles disponibles avec 'gpt' : {len([m for m in models_data.get('data', []) if 'gpt' in m.get('id', '').lower()])}")
    else:
        print(f"⚠️  Connexion API: HTTP {response.status_code}")
except Exception as e:
    print(f"❌ Erreur connexion: {str(e)[:100]}...")
    
print(f"\n🎯 Modèle sélectionné: {model_name}")
print(f"⚙️  Paramètres: Temperature={temperature}, Max tokens={max_tokens}")
print(f"📊 Mode d'analyse: {analysis_mode}")

In [ ]:
# Fonctions utilitaires pour traitement d'images
def encode_image_base64(image_path: Union[str, Path]) -> str:
    """
    Encode une image locale en base64 pour GPT-5.
    
    Args:
        image_path: Chemin vers l'image locale
        
    Returns:
        String base64 de l'image
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        raise ValueError(f"Erreur lecture image {image_path}: {str(e)}")

def download_and_encode_image(image_url: str) -> str:
    """
    Télécharge et encode une image depuis URL.
    
    Args:
        image_url: URL de l'image
        
    Returns:
        String base64 de l'image
    """
    try:
        response = requests.get(image_url, timeout=30)
        response.raise_for_status()
        return base64.b64encode(response.content).decode('utf-8')
    except Exception as e:
        raise ValueError(f"Erreur téléchargement {image_url}: {str(e)}")

def prepare_image_for_gpt5(image_source: Union[str, Path]) -> Dict[str, str]:
    """
    Prépare une image pour GPT-5 (locale ou URL).
    
    Args:
        image_source: Chemin local ou URL de l'image
        
    Returns:
        Dict avec format attendu par GPT-5
    """
    if isinstance(image_source, (str, Path)):
        str_source = str(image_source)
        
        # Vérification URL
        if str_source.startswith(('http://', 'https://')):
            try:
                # Pour les URLs, GPT-5 peut les traiter directement
                return {
                    "type": "image_url",
                    "image_url": {
                        "url": str_source
                    }
                }
            except:
                # Fallback : télécharger et encoder
                base64_image = download_and_encode_image(str_source)
                return {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
        else:
            # Image locale
            if Path(str_source).exists():
                base64_image = encode_image_base64(str_source)
                return {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            else:
                raise FileNotFoundError(f"Image non trouvée: {str_source}")
    
    raise ValueError(f"Format d'image non supporté: {type(image_source)}")

print("✅ Fonctions utilitaires images prêtes")

In [ ]:
# Fonction principale d'analyse d'image avec GPT-5
def analyze_image_with_gpt5(image_source: Union[str, Path], 
                           prompt: str = None,
                           analysis_type: str = "detailed") -> Dict[str, Any]:
    """
    Analyse une image avec GPT-5 multimodal.
    
    Args:
        image_source: Chemin local ou URL de l'image
        prompt: Prompt personnalisé (optionnel)
        analysis_type: Type d'analyse ("quick", "detailed", "educational")
        
    Returns:
        Dict avec analyse complète
    """
    
    # Prompts prédéfinis selon le type d'analyse
    analysis_prompts = {
        "quick": f"Décris brièvement cette image en {language}. Sois concis mais précis.",
        
        "detailed": f"""Analyse cette image en détail en {language}. Inclus :
        
1. **Description générale** : Que voit-on dans l'image ?
2. **Éléments visuels** : Couleurs, composition, style artistique
3. **Contexte** : Époque, lieu, situation probable
4. **Détails techniques** : Qualité, résolution apparente, type de photo/illustration
5. **Émotions/Atmosphère** : Quelle ambiance dégage l'image ?
6. **Interprétation** : Signification possible, message artistique

Sois précis et pédagogique dans tes explications.""",
        
        "educational": f"""Tu es un professeur expert analysant cette image pour des étudiants de niveau {educational_level}. En {language}, fournis :

🎯 **ANALYSE PÉDAGOGIQUE**

**1. Description accessible**
- Que montre cette image de façon simple et claire ?

**2. Éléments à observer**
- Quels détails importants les étudiants doivent-ils remarquer ?
- Techniques artistiques ou photographiques utilisées

**3. Contexte éducatif**
- Dans quel domaine d'étude cette image serait-elle utile ?
- Quelles disciplines académiques peuvent l'utiliser ?

**4. Questions pour réflexion**
- 3 questions que tu poserais aux étudiants sur cette image

**5. Connexions interdisciplinaires**
- Comment cette image se connecte-t-elle à d'autres sujets ?

Adapte ton vocabulaire au niveau {educational_level}."""
    }
    
    # Sélection du prompt
    if prompt is None:
        prompt = analysis_prompts.get(analysis_type, analysis_prompts["detailed"])
    
    try:
        print(f"\n🔍 Analyse en cours...")
        print(f"📝 Type: {analysis_type}")
        print(f"🖼️  Source: {str(image_source)[:100]}{'...' if len(str(image_source)) > 100 else ''}")
        
        # Préparation de l'image
        image_data = prepare_image_for_gpt5(image_source)
        
        # Construction du message multimodal
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    image_data
                ]
            }
        ]
        
        # Payload de la requête
        payload = {
            "model": model_name,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p
        }
        
        # Requête API
        start_time = datetime.now()
        response = requests.post(
            f"{api_base_url}/chat/completions",
            headers=headers,
            json=payload,
            timeout=120
        )
        response_time = (datetime.now() - start_time).total_seconds()
        
        if response.status_code == 200:
            result = response.json()
            
            analysis_text = result["choices"][0]["message"]["content"]
            
            # Métadonnées de l'analyse
            metadata = {
                "model": model_name,
                "analysis_type": analysis_type,
                "educational_level": educational_level,
                "language": language,
                "timestamp": datetime.now().isoformat(),
                "response_time": response_time,
                "tokens_used": result.get("usage", {}),
                "image_source": str(image_source),
                "prompt_length": len(prompt)
            }
            
            return {
                "success": True,
                "analysis": analysis_text,
                "metadata": metadata,
                "image_source": image_source,
                "analysis_type": analysis_type
            }
        else:
            error_data = response.json() if response.headers.get('content-type', '').startswith('application/json') else {}
            error_msg = error_data.get("error", {}).get("message", f"HTTP {response.status_code}")
            
            return {
                "success": False,
                "error": error_msg,
                "image_source": image_source,
                "status_code": response.status_code
            }
            
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "image_source": image_source
        }

print("✅ Fonction d'analyse GPT-5 prête")

In [ ]:
# Exemples d'images pour démonstration
print("\n🖼️  EXEMPLES D'ANALYSE GPT-5")
print("=" * 40)

# Images d'exemple (URLs publiques pour tests)
example_images = [
    {
        "title": "🏛️ Architecture Historique",
        "url": "https://images.unsplash.com/photo-1539037116277-4db20889f2d4?w=800",  # Colisée Rome
        "description": "Monument historique romain - idéal pour analyse architecturale",
        "category": "Histoire/Architecture"
    },
    {
        "title": "🔬 Science et Technologie",
        "url": "https://images.unsplash.com/photo-1532094349884-543bc11b234d?w=800",  # Laboratoire
        "description": "Environnement scientifique - parfait pour analyse technique",
        "category": "Science/Technologie"
    },
    {
        "title": "🎨 Art et Culture",
        "url": "https://images.unsplash.com/photo-1541961017774-22349e4a1262?w=800",  # Peinture
        "description": "Œuvre artistique - excellent pour analyse esthétique",
        "category": "Art/Culture"
    },
    {
        "title": "🌍 Nature et Environnement",
        "url": "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=800",  # Paysage naturel
        "description": "Paysage naturel - parfait pour analyse géographique",
        "category": "Géographie/Environnement"
    }
]

# Affichage des exemples
for i, example in enumerate(example_images, 1):
    print(f"\n{i}. {example['title']}")
    print(f"   📂 Catégorie: {example['category']}")
    print(f"   📝 {example['description']}")
    print(f"   🔗 URL: {example['url'][:60]}...")

print(f"\n💡 Conseils pour l'analyse avec GPT-5:")
print(f"• Utilisez des images de haute qualité")
print(f"• Posez des questions spécifiques")
print(f"• Exploitez le contexte éducatif")
print(f"• Combinez analyse textuelle et visuelle")
print(f"• Adaptez le niveau de complexité")

In [ ]:
# Analyse d'une image de démonstration
print("\n🚀 ANALYSE DE DÉMONSTRATION - GPT-5")
print("=" * 50)

# Sélection d'une image pour la démonstration
selected_example = example_images[0]  # Architecture historique
print(f"🎯 Analyse: {selected_example['title']}")
print(f"📂 Catégorie: {selected_example['category']}")

# Test de l'analyse avec les différents modes
analysis_results = []

for mode in ['quick', 'detailed', 'educational']:
    print(f"\n🔍 Mode d'analyse: {mode.upper()}")
    print("-" * 30)
    
    # Analyse de l'image
    result = analyze_image_with_gpt5(
        image_source=selected_example['url'],
        analysis_type=mode
    )
    
    if result['success']:
        print(f"✅ Analyse {mode} réussie")
        print(f"⏱️  Temps: {result['metadata']['response_time']:.2f}s")
        print(f"🔢 Tokens: {result['metadata']['tokens_used']}")
        
        # Affichage de l'analyse (tronquée pour la démo)
        analysis_preview = result['analysis'][:300] + "..." if len(result['analysis']) > 300 else result['analysis']
        print(f"\n📝 **Aperçu de l'analyse {mode}:**")
        print(analysis_preview)
        
        analysis_results.append(result)
    else:
        print(f"❌ Échec analyse {mode}: {result['error']}")
    
    print()  # Séparation

# Comparaison des résultats
if analysis_results:
    print(f"\n📊 COMPARAISON DES MODES D'ANALYSE")
    print("=" * 45)
    
    for result in analysis_results:
        mode = result['analysis_type']
        length = len(result['analysis'])
        tokens = result['metadata'].get('tokens_used', {}).get('total_tokens', 'N/A')
        time = result['metadata']['response_time']
        
        print(f"{mode.capitalize():12} | {length:4d} chars | {tokens:>6} tokens | {time:5.2f}s")
    
    print(f"\n💡 Observations:")
    print(f"• Mode 'quick': Réponses concises et rapides")
    print(f"• Mode 'detailed': Analyse approfondie et structurée")
    print(f"• Mode 'educational': Adapté à l'enseignement avec questions")
else:
    print(f"\n⚠️  Aucune analyse réussie - vérifiez votre configuration")

In [ ]:
# Génération de descriptions d'accessibilité (Alt text)
if generate_alt_text and analysis_results:
    print("\n♿ GÉNÉRATION DE DESCRIPTIONS D'ACCESSIBILITÉ")
    print("=" * 55)
    
    # Prompt spécialisé pour l'accessibilité
    accessibility_prompt = f"""Génère une description d'accessibilité (alt text) pour cette image en {language}.
    
Critères :
- Maximum 125 caractères
- Description factuelle et objective
- Inclut les éléments essentiels pour la compréhension
- Évite les interprétations subjectives
- Adapté aux lecteurs d'écran

Format : Fournis UNIQUEMENT la description, sans formatage supplémentaire."""
    
    # Génération de l'alt text
    alt_result = analyze_image_with_gpt5(
        image_source=selected_example['url'],
        prompt=accessibility_prompt
    )
    
    if alt_result['success']:
        alt_text = alt_result['analysis'].strip()
        
        print(f"✅ Description d'accessibilité générée")
        print(f"📝 Alt text ({len(alt_text)} caractères) :")
        print(f'   "{alt_text}"')
        
        if len(alt_text) > 125:
            print(f"⚠️  Longueur dépassée ({len(alt_text)}/125 chars) - considérez une version plus courte")
        else:
            print(f"✅ Longueur optimale ({len(alt_text)}/125 chars)")
    else:
        print(f"❌ Erreur génération alt text: {alt_result['error']}")
else:
    print(f"\n⏭️  Génération alt text désactivée ou pas d'analyse disponible")

In [ ]:
# Mode interactif - Analyse d'image personnalisée
if notebook_mode == "interactive" and not skip_widgets:
    print("\n🎨 MODE INTERACTIF - ANALYSE PERSONNALISÉE")
    print("=" * 55)
    
    print("\n💡 Analysez votre propre image avec GPT-5:")
    print("Formats supportés: URL https:// ou chemin local")
    print("(Laissez vide pour passer à la suite)")
    
    try:
        user_image = input("\n🖼️  URL ou chemin de votre image: ").strip()
        
        if user_image:
            # Paramètres d'analyse personnalisés
            print("\n⚙️  Paramètres d'analyse (appuyez Entrée pour défaut):")
            custom_mode = input(f"📊 Mode [{analysis_mode}]: ").strip() or analysis_mode
            custom_prompt = input("📝 Prompt personnalisé (optionnel): ").strip()
            
            print(f"\n🔍 Analyse de votre image en cours...")
            
            # Analyse personnalisée
            if custom_prompt:
                user_result = analyze_image_with_gpt5(
                    image_source=user_image,
                    prompt=custom_prompt
                )
            else:
                user_result = analyze_image_with_gpt5(
                    image_source=user_image,
                    analysis_type=custom_mode
                )
            
            if user_result['success']:
                print(f"\n🎉 Analyse réussie!")
                print(f"⏱️  Temps: {user_result['metadata']['response_time']:.2f}s")
                print(f"\n📝 **Analyse GPT-5:**")
                print(user_result['analysis'])
                
                # Option de sauvegarde
                if export_analysis:
                    save_choice = input("\n💾 Sauvegarder cette analyse ? (o/N): ").strip().lower()
                    if save_choice in ['o', 'oui', 'y', 'yes']:
                        output_dir = GENAI_ROOT / 'outputs' / 'gpt5_analysis'
                        output_dir.mkdir(parents=True, exist_ok=True)
                        
                        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                        analysis_file = output_dir / f"gpt5_analysis_{timestamp}.json"
                        
                        with open(analysis_file, 'w', encoding='utf-8') as f:
                            json.dump(user_result, f, indent=2, ensure_ascii=False)
                        
                        print(f"💾 Analyse sauvegardée: {analysis_file}")
            else:
                print(f"\n❌ Erreur: {user_result['error']}")
                print(f"🔍 Source: {user_result['image_source']}")
        else:
            print("\n⏭️  Mode interactif ignoré")
            
    except (KeyboardInterrupt, EOFError):
        print("\n⏭️  Mode interactif interrompu")
else:
    print("\n🤖 Mode batch - Interface interactive désactivée")
    print("💡 Pour mode interactif: notebook_mode = 'interactive'")

In [ ]:
# Cas d'usage pédagogiques avec GPT-5
print("\n🎓 CAS D'USAGE PÉDAGOGIQUES GPT-5")
print("=" * 45)

# Exemples d'applications éducatives
educational_use_cases = {
    "Histoire": {
        "description": "Analyse de documents historiques, œuvres d'art, monuments",
        "exemple": "Analyser une fresque renaissance pour comprendre le contexte social",
        "prompt_template": "Analyse cette image historique pour des étudiants en histoire. Explique le contexte historique, les éléments symboliques, et l'importance culturelle."
    },
    
    "Sciences": {
        "description": "Analyse d'expériences, schémas scientifiques, phénomènes naturels",
        "exemple": "Expliquer un diagramme de cellule ou une réaction chimique",
        "prompt_template": "En tant qu'enseignant de sciences, explique cette image scientifique. Identifie les éléments techniques et leur fonctionnement."
    },
    
    "Géographie": {
        "description": "Étude de paysages, cartes, phénomènes géologiques",
        "exemple": "Analyser une photo satellite ou un paysage géographique",
        "prompt_template": "Analyse cette image géographique pour des étudiants. Explique les formations géologiques, le climat, et l'impact humain visible."
    },
    
    "Art et Culture": {
        "description": "Critique artistique, analyse stylistique, histoire de l'art",
        "exemple": "Comprendre les techniques d'un tableau impressionniste",
        "prompt_template": "Fais une analyse artistique de cette œuvre pour des étudiants en art. Inclus style, techniques, et signification culturelle."
    },
    
    "Médecine": {
        "description": "Analyse d'imagerie médicale, anatomie, cas cliniques",
        "exemple": "Expliquer une radiographie ou un schéma anatomique",
        "prompt_template": "Analyse cette image médicale pour des étudiants en médecine. Explique l'anatomie visible et les points d'intérêt clinique."
    }
}

# Affichage des cas d'usage
for domain, info in educational_use_cases.items():
    print(f"\n📚 **{domain}**")
    print(f"   📋 {info['description']}")
    print(f"   💡 Exemple: {info['exemple']}")
    print(f"   📝 Template de prompt disponible")

print(f"\n🎯 Avantages GPT-5 pour l'éducation:")
print(f"• **Multimodal** : Analyse texte + image simultanée")
print(f"• **Contextuel** : Comprend le niveau éducatif")
print(f"• **Adaptable** : Ajuste le vocabulaire selon l'audience")
print(f"• **Interactif** : Permet les questions de suivi")
print(f"• **Précis** : Analyse détaillée et factuelle")

print(f"\n📝 Exemple de workflow pédagogique:")
print(f"1. Sélection d'image pertinente au cours")
print(f"2. Analyse GPT-5 avec prompt éducatif")
print(f"3. Génération de questions pour les étudiants")
print(f"4. Discussion interactive basée sur l'analyse")
print(f"5. Évaluation de la compréhension")

## 🎯 Résumé et Bonnes Pratiques

### ✅ Ce que vous avez appris

- [ ] **Configuration GPT-5** : OpenRouter API et paramètres optimaux
- [ ] **Analyse multimodale** : Combinaison texte + image
- [ ] **Prompts éducatifs** : Adaptation au niveau d'enseignement
- [ ] **Cas d'usage pédagogiques** : Applications concrètes par domaine
- [ ] **Optimisation** : Paramètres de qualité et performance

### 🚀 Prochaines étapes

1. **Expérimentez** avec vos propres images éducatives
2. **Testez** différents niveaux d'analyse selon votre public
3. **Intégrez** GPT-5 dans vos workflows pédagogiques
4. **Combinez** avec DALL-E 3 pour génération + analyse
5. **Explorez** les notebooks avancés (Module 02)

### 💡 Conseils pour l'utilisation

**✅ Bonnes pratiques:**
- Utilisez des images de haute qualité
- Adaptez le prompt au niveau éducatif
- Combinez analyse visuelle et contextuelle
- Générez des questions pédagogiques
- Sauvegardez les analyses réussies

**❌ Évitez:**
- Images trop petites ou de mauvaise qualité
- Prompts vagues ou génériques
- Oublier le contexte éducatif
- Ignorer les limitations du modèle
- Ne pas vérifier la factualité

### 🔗 Ressources complémentaires

- **Documentation OpenRouter** : [openrouter.ai](https://openrouter.ai)
- **Guide GPT-5** : Capacités multimodales
- **Templates éducatifs** : `docs/genai-phase2-templates.md`
- **Standards CoursIA** : `docs/genai-images-development-standards.md`